[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_elec_stats/EMQA_elec_stats.ipynb)

# EMQA_elec_stats

German electricity price statistics.

**Output:** `elec_statistics.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import os
def load_elec():
    paths = [
        '../../charts/electricity_cache.csv',
        '/Users/danielpele/Documents/Energy MBA/charts/electricity_cache.csv',
    ]
    for p in paths:
        if os.path.exists(p):
            df = pd.read_csv(p, parse_dates=[0], index_col=0)
            return df
    # Fallback: generate from German data
    paths2 = [
        '../../charts/german_dayahead_prices.csv',
        '/Users/danielpele/Documents/Energy MBA/charts/german_dayahead_prices.csv',
    ]
    for p in paths2:
        if os.path.exists(p):
            return pd.read_csv(p, parse_dates=[0], index_col=0)
    raise FileNotFoundError("No electricity data found")

elec = load_elec()
# Get price column
if 'price' in elec.columns:
    price_col = 'price'
elif 'Price' in elec.columns:
    price_col = 'Price'
else:
    price_col = elec.columns[0]
elec_price = elec[price_col].dropna()


In [ ]:
print("=== German Electricity Statistics ===")
print(f"Mean: {elec_price.mean():.2f} EUR/MWh")
print(f"Median: {elec_price.median():.2f}")
print(f"Std: {elec_price.std():.2f}")
print(f"Min: {elec_price.min():.2f}")
print(f"Max: {elec_price.max():.2f}")
print(f"Negative hours: {(elec_price<0).sum()} ({(elec_price<0).mean()*100:.1f}%)")

fig, ax = plt.subplots(figsize=(12, 5))
ax.hist(elec_price.clip(-100, 500), bins=100, color=COLORS['blue'], alpha=0.7, edgecolor='white')
ax.axvline(0, color=COLORS['red'], ls='--', lw=1.5)
ax.axvline(elec_price.mean(), color=COLORS['orange'], ls='--', lw=1.5)
ax.set_title('German Electricity Price Distribution', fontweight='bold')
ax.set_xlabel('EUR/MWh')
ax.legend(['Zero', f'Mean ({elec_price.mean():.0f})', 'Distribution'],
          loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=3, frameon=False)
fig.tight_layout()
save_fig(fig, 'elec_statistics.pdf')
plt.show()
